In [2]:
import numpy as np
import itertools
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
%matplotlib inline
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [3]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [4]:
train_path = 'RND/train'
valid_path = 'RND/valid'
dirs = list(map(str, range(1, 10 + 1)))

In [5]:
train_batches = ImageDataGenerator(rescale=1./255,
                                   rotation_range=8,
                                   width_shift_range=0.08,
                                   shear_range=0.3,
                                   height_shift_range=0.08,
                                   zoom_range=0.08).flow_from_directory(train_path, target_size=(28, 28),
                                                        classes=dirs, batch_size=10, color_mode='grayscale')
valid_batches = ImageDataGenerator(rescale=1./255,
                                   rotation_range=8,
                                   width_shift_range=0.08,
                                   shear_range=0.3,
                                   height_shift_range=0.08,
                                   zoom_range=0.08).flow_from_directory(valid_path, target_size=(28, 28),
                                                        classes=dirs, batch_size=10, color_mode='grayscale')

Found 400 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


In [223]:
def baseline_model():
    # create model
    model = keras.applications.vgg16.VGG16()
#     model = Sequential()
#     model.add(Conv2D(32, (3, 3), input_shape=(1, 28, 28), activation='relu'))
#     model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))
#     model.add(Flatten())
#     model.add(Dense(128, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(10, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [6]:
def baseline_model1():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=(1, 28, 28)))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64,(3, 3)))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())

    # Fully connected layer
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10))

    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
# build the model
model = baseline_model1()

In [9]:
# Fit the model
model.fit_generator(train_batches, steps_per_epoch=40, 
                    validation_data=valid_batches, validation_steps=10, epochs=30, verbose=2)

Epoch 1/30
 - 7s - loss: 0.2276 - acc: 0.9300 - val_loss: 0.3083 - val_acc: 0.9000
Epoch 2/30
 - 7s - loss: 0.1736 - acc: 0.9625 - val_loss: 0.4936 - val_acc: 0.8300
Epoch 3/30
 - 7s - loss: 0.1846 - acc: 0.9525 - val_loss: 0.2705 - val_acc: 0.9200
Epoch 4/30
 - 7s - loss: 0.1933 - acc: 0.9425 - val_loss: 0.3145 - val_acc: 0.9200
Epoch 5/30
 - 7s - loss: 0.2989 - acc: 0.9050 - val_loss: 0.4779 - val_acc: 0.8400
Epoch 6/30
 - 8s - loss: 0.2220 - acc: 0.9475 - val_loss: 0.3718 - val_acc: 0.8900
Epoch 7/30
 - 7s - loss: 0.2185 - acc: 0.9350 - val_loss: 0.4804 - val_acc: 0.9100
Epoch 8/30
 - 7s - loss: 0.2046 - acc: 0.9300 - val_loss: 0.5225 - val_acc: 0.8600
Epoch 9/30
 - 7s - loss: 0.2073 - acc: 0.9375 - val_loss: 0.3074 - val_acc: 0.9100
Epoch 10/30
 - 7s - loss: 0.1859 - acc: 0.9450 - val_loss: 0.2866 - val_acc: 0.9200
Epoch 11/30
 - 7s - loss: 0.2030 - acc: 0.9575 - val_loss: 0.4350 - val_acc: 0.8600
Epoch 12/30
 - 7s - loss: 0.1614 - acc: 0.9525 - val_loss: 0.2916 - val_acc: 0.8900
E

In [10]:
test_batches = ImageDataGenerator(rescale=1./255,
                                   rotation_range=8,
                                   width_shift_range=0.08,
                                   shear_range=0.3,
                                   height_shift_range=0.08,
                                   zoom_range=0.08).flow_from_directory('RND/test', target_size=(28, 28),
                                                        classes=dirs, batch_size=20, color_mode='grayscale', shuffle=False)

Found 19 images belonging to 10 classes.


In [11]:
predictions = model.predict_generator(test_batches, steps=1, verbose=0)

In [12]:
np.array(list(map(np.argmax, predictions))) + 1

array([ 1,  1,  2,  2,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9,
       10, 10])

In [13]:
model.save_weights('weights/conv1.ckpt')